In [3]:
import pandas as pd
from sqlalchemy import create_engine,  MetaData, Table, Column, Date, String, Boolean

import datetime

## Ouverture des csv en dataframe

### aeronefs

In [4]:
# ouvrir csv pour mettre en data frame
df_aeronef = pd.read_csv(r'dataset\aeronefs_clean.csv')

In [5]:
# convertir les données avant insertion dans la table sql
df_aeronef['debut_service'] = pd.to_datetime(df_aeronef['debut_service'])
df_aeronef['last_maint'] = pd.to_datetime(df_aeronef['last_maint'])
df_aeronef['end_maint'] = pd.to_datetime(df_aeronef['end_maint'])

df_aeronef['en_maintenance'] = df_aeronef['en_maintenance'].astype('boolean')


In [6]:
df_aeronef

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,E170_6356,E170,2000-03-17,2024-03-27,False,NaT
1,B767_2803,B767,2020-12-03,2024-06-01,False,NaT
2,CRJ700_6943,CRJ700,2015-01-29,2024-06-02,True,2024-06-03 14:13:00
3,A350_1177,A350,2005-08-07,2024-01-04,False,NaT
4,B777_4029,B777,2009-01-01,2024-06-01,False,NaT
...,...,...,...,...,...,...
227,B737_4369,B737,2010-01-05,2024-06-01,False,NaT
228,A340_4021,A340,2004-12-13,2024-05-31,False,NaT
229,CRJ900_1740,CRJ900,2003-07-20,2024-06-01,False,NaT
230,E170_5551,E170,2021-04-10,2024-06-01,False,NaT


### Composants

In [ ]:
# ouvrir csv pour mettre en data frame
df_composants = pd.read_csv(r'dataset\composants_clean.csv')

In [ ]:
# convertir les données avant insertion dans la table sql
df_composants['debut_service'] = pd.to_datetime(df_composants['debut_service'])
df_composants['last_maint'] = pd.to_datetime(df_composants['last_maint'])
df_composants['end_maint'] = pd.to_datetime(df_composants['end_maint'])

df_composants['en_maintenance'] = df_composants['en_maintenance'].astype('boolean')


In [ ]:
df_aeronef

## Connexion et insertion des données dans la bdd

### Connexion à la bdd

In [7]:
# Définition des paramètres de connexion à la base de données
username = 'postgres'
password = 'admin'
host = 'localhost'
port = '5432'
dbname = 'preventive_maintenance'

# Configuration de la connexion à la base de données PostgreSQL
connection_str = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_str, echo = False)

# # Connexion à la base de données
# connection = engine.connect()

# generation de l'obet bdd
metadata = MetaData()
metadata.reflect(bind=engine)


### declaration des tables

In [84]:
aeronefs = Table(
    'aeronefs', metadata,
    )

composants = Table(
    'composants', metadata,
    )

degradations = Table(
    'degradations', metadata,
    )

logs_vols = Table(
    'logs_vols', metadata,
    )

metadata.create_all(engine)

### insertion des données

In [96]:

def maj_table(df:pd.DataFrame, table : Table)->None:  # , ref_temp: str, data_to:dict[str:str, str:Date, str:int]
    """
    Insère ou met à jour les données d'une table SQL à partir d'un DataFrame Pandas.

    Pour chaque ligne du DataFrame, la fonction vérifie si la référence (ref_aero) existe déjà dans la table.
    Si la référence n'existe pas, la ligne est insérée. Si la référence existe et que les données sont différentes,
    la ligne est mise à jour.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame contenant les données à insérer ou mettre à jour dans la table.
    table : Table
        Objet Table de SQLAlchemy représentant la table dans laquelle les données seront insérées ou mises à jour.
    """
    with engine.connect() as connection:
        
        comptage_insertion = 0
        comptage_update = 0

        # Parcour de l'ensemble des lignes du df à inserer
        for row in df.itertuples(index=False):

            data_to = {}
            
            # Récupération de la valeur de la colonne ref_aero
            ref_temp = row[0]

            # Préparation des données à inserer ou updater
            column_names = df.columns

            for index, col in zip(range(len(row)), column_names):

                if row[index]  is not pd.NaT:
                    data_to[col] = row[index]
                else : 
                    data_to[col] = None


            # identification si la ref_aero est déjà présente ou non dans la table 
            sel = table.select().where(table.c.ref_aero == ref_temp)
            result = connection.execute(sel)   
            row_in_table = result.fetchone()    

            if row_in_table is None: 
                
                #insertion de la ligne dans la table
                connection.execute(table.insert().values(data_to))
                comptage_insertion += 1

            else:
                # mise au même format de row et row_in_table pour les dates
                row1 = [i for i in row]
                row2 = [i for i in row_in_table]

                for i, elem in enumerate(row2):

                    if isinstance(elem, datetime.date) or elem is None:

                        row2[i] = pd.Timestamp(row2[i])

                # Ajout de la condition de doublon ou non
                if row1 != row2 and row1[0] == row2[0]:
                    # update de la ligne dans la table si non doublon
                    connection.execute(table.update().where(table.c.ref_aero == ref_temp).values(data_to))
                    # Information sur la mise à jour
                    print(f'Comparaison des données pour MàJ : \n{row1}\n{row2}\n')
                    
                    comptage_update += 1
                
            # Commit dans la table des insertions ou update 
            connection.commit()

    # Affichage des résultats
    print(f'Nombre de lignes insérées : {comptage_insertion}')
    print(f'Nombre de lignes mises à jour : {comptage_update}')


In [97]:
maj_table(df_aeronef, aeronefs)

Comparaison des données pour MàJ : 
['E170_5551', 'E170', Timestamp('2021-04-10 00:00:00'), Timestamp('2024-06-01 00:00:00'), False, NaT]
['E170_5551', 'E170', Timestamp('2017-02-07 00:00:00'), Timestamp('2024-06-01 00:00:00'), False, NaT]

Nombre de lignes insérées : 231
Nombre de lignes mises à jour : 1


In [ ]:
maj_table(df_composants, composants)